## Импорты

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu, shapiro, chi2_contingency, median_test, norm
from statsmodels.stats.power import GofChisquarePower


In [2]:
df = pd.read_csv('basic_eda_dataframe.csv', sep=';')

In [3]:
df.head(5)

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder,Systolic_pressure,Diastolic_pressure
0,Male,27,Software Engineer,6.1,6,42,6,Overweight,77,4200,No disorders,126,83
1,Male,28,Doctor,6.2,6,60,8,Normal,75,10000,No disorders,125,80
2,Male,28,Sales Representative,5.9,4,30,8,Obese,85,3000,Sleep Apnea,140,90
3,Male,28,Software Engineer,5.9,4,30,8,Obese,85,3000,Insomnia,140,90
4,Male,29,Teacher,6.3,6,40,7,Obese,82,3500,Insomnia,140,90


## Тест Шапиро-Уилка на проверку нормальности распределения

In [4]:
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    stat, p = shapiro(df[col])
    if p < 0.05:
        print(f'Для "{col}": p-value = {p:.7f} -> распределение НЕ нормальное')
        print('----------------------------------------------------------------------------')
    else:
        print(f'Для "{col}": p-value = {p:.7f} -> распределение нормальное')
        print('----------------------------------------------------------------------------')

Для "Age": p-value = 0.0001915 -> распределение НЕ нормальное
----------------------------------------------------------------------------
Для "Sleep Duration": p-value = 0.0000368 -> распределение НЕ нормальное
----------------------------------------------------------------------------
Для "Quality of Sleep": p-value = 0.0000005 -> распределение НЕ нормальное
----------------------------------------------------------------------------
Для "Physical Activity Level": p-value = 0.0000002 -> распределение НЕ нормальное
----------------------------------------------------------------------------
Для "Stress Level": p-value = 0.0000001 -> распределение НЕ нормальное
----------------------------------------------------------------------------
Для "Heart Rate": p-value = 0.0000001 -> распределение НЕ нормальное
----------------------------------------------------------------------------
Для "Daily Steps": p-value = 0.0001066 -> распределение НЕ нормальное
------------------------------------

## Критерий Манна-Уитни

### Проверки гипотез:

In [5]:
#Данные о людях с расстройствами сна
people_with_disorders = df.loc[df['Sleep Disorder'] != 'No disorders']

#Данные о людях без расстройств сна
people_without_disorders = df.loc[df['Sleep Disorder'] == 'No disorders']

#выборки с данными о продожительности сна
sleep_duration_with_disorders = people_with_disorders['Sleep Duration']
sleep_duration_without_disorders = people_without_disorders['Sleep Duration']

#выборки с данными о качестве сна
sleep_quality_with_disorders = people_with_disorders['Quality of Sleep']
sleep_quality_without_disorders = people_without_disorders['Quality of Sleep']

#выборки с данными о возрасте
age_with_disorders = people_with_disorders['Age']
age_without_disorders = people_without_disorders['Age']


#### Наличие разницы в продолжительности сна у людей с расстройствами сна и без:

**Нулевая гипотеза (H₀):** Продолжительность сна одинакова у людей с раcстройствами сна и без.

**Альтернативная гипозеза (H₁):** Связь существует. Продолжительность сна различается между людьми с расстройствами сна и без.

In [6]:
# Выполняем тест Манна-Уитни: продолжительность сна
U_stat, p_value = mannwhitneyu(sleep_duration_with_disorders, sleep_duration_without_disorders, 
                              alternative='two-sided',
                              method='exact')

print(f"Результаты критерия Манна-Уитни:")
print(f"U-статистика = {U_stat:.2f}")
print(f"p-value = {p_value:.5f}")

# Интерпретация p-value
print(f"\nИнтерпретация p-value:")
if p_value < 0.001:
    print(f"  p = {p_value:.5f} < 0.001 → различие ВЕСЬМА значимо")
elif p_value < 0.01:
    print(f"  p = {p_value:.5f} < 0.01 → различие ВЫСОКО значимо")
elif p_value < 0.05:
    print(f"  p = {p_value:.5f} < 0.05 → различие значимо")
else:
    print(f"  p = {p_value:.5f} ≥ 0.05 → различие НЕ значимо")


Результаты критерия Манна-Уитни:
U-статистика = 1408.00
p-value = 0.00057

Интерпретация p-value:
  p = 0.00057 < 0.001 → различие ВЕСЬМА значимо


In [7]:
print("РАЗМЕР ЭФФЕКТА: продолжительность сна")
print('-'*40)

# Расчет размера эффекта r для Манна-Уитни
def mann_whitney_effect_size(x, y):
    n1, n2 = len(x), len(y)
    U = min(mannwhitneyu(x, y, alternative='two-sided')[0],
            mannwhitneyu(y, x, alternative='two-sided')[0])
    
    
    r = 1 - (2 * U) / (n1 * n2)
    return r


r1 = mann_whitney_effect_size(sleep_duration_with_disorders, sleep_duration_without_disorders)

print(f"Размер эффекта: {r1:.3f}")

# Интерпретация размера эффекта
print(f"\nИнтерпретация размера эффекта |r|:")
effect_size = abs(r1)
if effect_size < 0.1:
    print(f"  |r| = {effect_size:.3f} < 0.1 → очень слабый эффект")
    interpretation = "очень слабый"
elif effect_size < 0.3:
    print(f"  |r| = {effect_size:.3f} < 0.3 → слабый эффект")
    interpretation = "слабый"
elif effect_size < 0.5:
    print(f"  |r| = {effect_size:.3f} < 0.5 → умеренный эффект")
    interpretation = "умеренный"
else:
    print(f"  |r| = {effect_size:.3f} ≥ 0.5 → сильный эффект")
    interpretation = "сильный"

РАЗМЕР ЭФФЕКТА: продолжительность сна
----------------------------------------
Размер эффекта: 0.346

Интерпретация размера эффекта |r|:
  |r| = 0.346 < 0.5 → умеренный эффект


#### Наличие разницы в качестве сна у людей с расстройствами сна и без:

**Нулевая гипотеза (H₀):** Качество сна одинаково у людей с расстройствами сна и без.

**Альтернативная гипозеза (H₁):** Связь существует. Качество сна различается между людьми с расстройствами сна и без.

In [8]:
# Выполняем тест Манна-Уитни: качество сна
U_stat, p_value = mannwhitneyu(sleep_quality_with_disorders, sleep_quality_without_disorders, 
                              alternative='two-sided',
                              method='exact')

print(f"Результаты критерия Манна-Уитни:")
print(f"U-статистика = {U_stat:.2f}")
print(f"p-value = {p_value:.5f}")

# Интерпретация p-value
print(f"\nИнтерпретация p-value:")
if p_value < 0.001:
    print(f"  p = {p_value:.5f} < 0.001 → различие ВЕСЬМА значимо")
elif p_value < 0.01:
    print(f"  p = {p_value:.5f} < 0.01 → различие ВЫСОКО значимо")
elif p_value < 0.05:
    print(f"  p = {p_value:.5f} < 0.05 → различие значимо")
else:
    print(f"  p = {p_value:.5f} ≥ 0.05 → различие НЕ значимо")



Результаты критерия Манна-Уитни:
U-статистика = 1392.50
p-value = 0.00044

Интерпретация p-value:
  p = 0.00044 < 0.001 → различие ВЕСЬМА значимо


In [9]:
print("РАЗМЕР ЭФФЕКТА: качество сна")
print('-'*40)

# Расчет размера эффекта r для Манна-Уитни
def mann_whitney_effect_size(x, y):
    n1, n2 = len(x), len(y)
    U = min(mannwhitneyu(x, y, alternative='two-sided')[0],
            mannwhitneyu(y, x, alternative='two-sided')[0])
    
    
    r = 1 - (2 * U) / (n1 * n2)
    return r


r1 = mann_whitney_effect_size(sleep_quality_with_disorders, sleep_quality_without_disorders)

print(f"Размер эффекта: {r1:.3f}")

# Интерпретация размера эффекта
print(f"\nИнтерпретация размера эффекта |r|:")
effect_size = abs(r1)
if effect_size < 0.1:
    print(f"  |r| = {effect_size:.3f} < 0.1 → очень слабый эффект")
    interpretation = "очень слабый"
elif effect_size < 0.3:
    print(f"  |r| = {effect_size:.3f} < 0.3 → слабый эффект")
    interpretation = "слабый"
elif effect_size < 0.5:
    print(f"  |r| = {effect_size:.3f} < 0.5 → умеренный эффект")
    interpretation = "умеренный"
else:
    print(f"  |r| = {effect_size:.3f} ≥ 0.5 → сильный эффект")
    interpretation = "сильный"

РАЗМЕР ЭФФЕКТА: качество сна
----------------------------------------
Размер эффекта: 0.353

Интерпретация размера эффекта |r|:
  |r| = 0.353 < 0.5 → умеренный эффект


#### Различие медианного возраста у людей с расстройствами сна и без:

**Нулевая гипотеза (H₀):** Медианный возраст одинаков у людей с расстройствами сна и без.

**Альтернативная гипозеза (H₁):** Связь существует. Медианный возраст различается между людьми с расстройствами сна и без.

In [10]:
# Выполняем тест Манна-Уитни: возраст
U_stat, p_value = mannwhitneyu(age_with_disorders, age_without_disorders, 
                              alternative='two-sided',
                              method='exact')

print(f"Результаты критерия Манна-Уитни:")
print(f"U-статистика = {U_stat:.2f}")
print(f"p-value = {p_value:.5f}")

# Интерпретация p-value
print(f"\nИнтерпретация p-value:")
if p_value < 0.001:
    print(f"  p = {p_value:.5f} < 0.001 → различие ВЕСЬМА значимо")
elif p_value < 0.01:
    print(f"  p = {p_value:.5f} < 0.01 → различие ВЫСОКО значимо")
elif p_value < 0.05:
    print(f"  p = {p_value:.5f} < 0.05 → различие значимо")
else:
    print(f"  p = {p_value:.5f} ≥ 0.05 → различие НЕ значимо")



Результаты критерия Манна-Уитни:
U-статистика = 2692.00
p-value = 0.01346

Интерпретация p-value:
  p = 0.01346 < 0.05 → различие значимо


In [11]:
print("РАЗМЕР ЭФФЕКТА: возраст")
print('-'*40)

# Расчет размера эффекта r для Манна-Уитни
def mann_whitney_effect_size(x, y):
    n1, n2 = len(x), len(y)
    U = min(mannwhitneyu(x, y, alternative='two-sided')[0],
            mannwhitneyu(y, x, alternative='two-sided')[0])
    
    
    r = 1 - (2 * U) / (n1 * n2)
    return r


r1 = mann_whitney_effect_size(age_with_disorders, age_without_disorders)

print(f"Размер эффекта: {r1:.3f}")

# Интерпретация размера эффекта
print(f"\nИнтерпретация размера эффекта |r|:")
effect_size = abs(r1)
if effect_size < 0.1:
    print(f"  |r| = {effect_size:.3f} < 0.1 → очень слабый эффект")
    interpretation = "очень слабый"
elif effect_size < 0.3:
    print(f"  |r| = {effect_size:.3f} < 0.3 → слабый эффект")
    interpretation = "слабый"
elif effect_size < 0.5:
    print(f"  |r| = {effect_size:.3f} < 0.5 → умеренный эффект")
    interpretation = "умеренный"
else:
    print(f"  |r| = {effect_size:.3f} ≥ 0.5 → сильный эффект")
    interpretation = "сильный"


РАЗМЕР ЭФФЕКТА: возраст
----------------------------------------
Размер эффекта: 0.250

Интерпретация размера эффекта |r|:
  |r| = 0.250 < 0.3 → слабый эффект


## Xи-квадрат

### Проверки гипотез:

In [12]:
bmi_category_and_sleep_disorders = df[['BMI Category', 'Sleep Disorder']].copy()
bmi_category_and_sleep_disorders.head()

,BMI Category,Sleep Disorder
0,Overweight,No disorders
1,Normal,No disorders
2,Obese,Sleep Apnea
3,Obese,Insomnia
4,Obese,Insomnia


In [13]:
bmi_category_and_sleep_disorders.loc[bmi_category_and_sleep_disorders['Sleep Disorder'] != 'No disorders', 'Sleep Disorder'] = 'Have a sleep disorder'
bmi_category_and_sleep_disorders.head(3)

,BMI Category,Sleep Disorder
0,Overweight,No disorders
1,Normal,No disorders
2,Obese,Have a sleep disorder


In [14]:
bmi_category_and_sleep_disorders.loc[bmi_category_and_sleep_disorders['BMI Category'] == 'Obese', 'BMI Category'] = 'Overweight'
bmi_category_and_sleep_disorders.head(3)

,BMI Category,Sleep Disorder
0,Overweight,No disorders
1,Normal,No disorders
2,Overweight,Have a sleep disorder


#### Связь между весовой категорией и наличием расстройств сна:

**Нулевая гипотеза (H₀):** Нет связи между весовой категорией и наличием расстройств сна. Вес и нарушения сна независимы. Распределение расстройств сна одинаково в обеих весовых группах.

**Альтернативная гипозеза (H₁):** Связь существует. Распределение расстройств сна различается между людьми с нормальным и избыточным весом.

In [15]:
#Подсчет наблюдаемых частот
data = pd.crosstab(bmi_category_and_sleep_disorders['Sleep Disorder'], bmi_category_and_sleep_disorders['BMI Category'])
print("Наблюдаемые частоты:")
print(data)

Наблюдаемые частоты:
BMI Category           Normal  Overweight
Sleep Disorder                           
Have a sleep disorder      15          44
No disorders               58          15


In [16]:
# Получаем таблицу сопряженности
observed = data.values

# Проверка гипотезы
chi2, p, dof, expected = chi2_contingency(observed)

# Вывод результатов
print("Статистика хи-квадрат:", round(chi2, 3))  # 13.393
print("p-значение:", round(p, 4))                # 0.004
print("Степени свободы:", dof)                   # 3
print("\nОжидаемые частоты (если H0 верна):\n", np.round(expected, 2))

# Решающее правило (alpha = 0.05)
alpha = 0.05
if p < alpha:
    print("\nВывод: p < alpha → отвергаем H0. Есть статистически значимая зависимость между весовой категорией и наличием расстройств сна.")
else:
    print("\nВывод: p >= alpha → не отвергаем H0. Зависимость не обнаружена.")

Статистика хи-квадрат: 36.377
p-значение: 0.0
Степени свободы: 1

Ожидаемые частоты (если H0 верна):
 [[32.63 26.37]
 [40.37 32.63]]

Вывод: p < alpha → отвергаем H0. Есть статистически значимая зависимость между весовой категорией и наличием расстройств сна.


In [17]:
#Расчет размера эффекта
n = observed.sum()
phi = np.sqrt(chi2 / n)
print(f"\n РАЗМЕР ЭФФЕКТА (Cohen’s) (φ-коэффициент):")
print(f"   φ = {phi:.3f}")
print("   Интерпретация φ:")
print("     0.10 - слабая связь")
print("     0.30 - умеренная связь")
print("     0.50 - сильная связь")
print("\n")


#Мощность теста — это вероятность правильно отвергнуть нулевую гипотезу, когда она на самом деле неверна. Это способность теста обнаружить эффект, если он действительно существует.
# Создаем объект для расчета мощности
power_analyzer = GofChisquarePower()

# Расчет мощности (аргументы в правильном порядке: effect_size, nobs, alpha, df)

power = power_analyzer.power(effect_size=phi, nobs=n, alpha=0.05, n_bins=observed.size)

print(f"\n МОЩНОСТЬ ТЕСТА:{power:.4f}")
print("   Интерпретация мощности теста:")
print("     < 0.6 (60%) — неадекватная мощность, результаты ненадежны")
print("     0.6-0.7 — низкая мощность")
print("     0.7-0.8 — приемлемая мощность")
print("     ≥ 0.8 — хорошая мощность (стандарт в большинстве исследований)")
print("     0≥ 0.9 — высокая мощность (требуется в клинических испытаниях)")



 РАЗМЕР ЭФФЕКТА (Cohen’s) (φ-коэффициент):
   φ = 0.525
   Интерпретация φ:
     0.10 - слабая связь
     0.30 - умеренная связь
     0.50 - сильная связь



 МОЩНОСТЬ ТЕСТА:0.9997
   Интерпретация мощности теста:
     < 0.6 (60%) — неадекватная мощность, результаты ненадежны
     0.6-0.7 — низкая мощность
     0.7-0.8 — приемлемая мощность
     ≥ 0.8 — хорошая мощность (стандарт в большинстве исследований)
     0≥ 0.9 — высокая мощность (требуется в клинических испытаниях)


In [18]:
print(f'p-value = {p:.4f},\npower(мощность) = {power:.4f}\n')
print('   Интерпретация результатов:')
print('     p < 0.05 & power < 0.6: Значимый результат, но низкая мощность — требуется подтверждение')
print('     p < 0.05 & power >= 0.8: Надежный значимый результат')
print('     p >= 0.05 & power < 0.6: Результат незначим, но тест мог пропустить существующий эффект')
print('     p >= 0.05 & power >= 0.6: Вероятно, эффекта действительно нет или он очень мал')
print('-'*95)

p-value = 0.0000,
power(мощность) = 0.9997

   Интерпретация результатов:
     p < 0.05 & power < 0.6: Значимый результат, но низкая мощность — требуется подтверждение
     p < 0.05 & power >= 0.8: Надежный значимый результат
     p >= 0.05 & power < 0.6: Результат незначим, но тест мог пропустить существующий эффект
     p >= 0.05 & power >= 0.6: Вероятно, эффекта действительно нет или он очень мал
-----------------------------------------------------------------------------------------------


#### Связь между гендером и наличием расстройств сна:

**Нулевая гипотеза (H₀):** Нет связи между гендером и наличием расстройств сна. Гендер и нарушения сна независимы. Распределение расстройств сна одинаково в обеих гендерных группах.

**Альтернативная гипозеза (H₁):** Связь существует. Распределение расстройств сна различается между женщинами и мужчинами.


In [19]:
gender_and_sleep_disorders = df[['Gender', 'Sleep Disorder']].copy()

In [20]:
gender_and_sleep_disorders.loc[gender_and_sleep_disorders['Sleep Disorder'] != 'No disorders', 'Sleep Disorder'] = 'Have a sleep disorder'

In [21]:
gender_and_sleep_disorders.sample(5)

,Gender,Sleep Disorder
112,Female,Have a sleep disorder
30,Female,No disorders
52,Female,No disorders
37,Male,No disorders
55,Male,No disorders


In [22]:
#Подсчет наблюдаемых частот
data1 = pd.crosstab(gender_and_sleep_disorders['Sleep Disorder'], gender_and_sleep_disorders['Gender'])
print("Наблюдаемые частоты:")
print(data1)

Наблюдаемые частоты:
Gender                 Female  Male
Sleep Disorder                     
Have a sleep disorder      34    25
No disorders               31    42


In [23]:
# Получаем таблицу сопряженности
observed = data1.values

# Проверка гипотезы
chi2, p, dof, expected = chi2_contingency(observed)

# Вывод результатов
print("Статистика хи-квадрат:", round(chi2, 3))  # 13.393
print("p-значение:", round(p, 4))                # 0.004
print("Степени свободы:", dof)                   # 3
print("\nОжидаемые частоты (если H0 верна):\n", np.round(expected, 2))

# Решающее правило (alpha = 0.05)
alpha = 0.05
if p < alpha:
    print("\nВывод: p < alpha → отвергаем H0. Есть статистически значимая зависимость между гендером и наличием расстройств сна.")
else:
    print("\nВывод: p >= alpha → не отвергаем H0. Зависимость не обнаружена.")

Статистика хи-квадрат: 2.425
p-значение: 0.1194
Степени свободы: 1

Ожидаемые частоты (если H0 верна):
 [[29.05 29.95]
 [35.95 37.05]]

Вывод: p >= alpha → не отвергаем H0. Зависимость не обнаружена.


In [24]:
#Расчет размера эффекта
n = observed.sum()
phi = np.sqrt(chi2 / n)
print(f"\n РАЗМЕР ЭФФЕКТА (Cohen’s) (φ-коэффициент):")
print(f"   φ = {phi:.3f}")
print("   Интерпретация φ:")
print("     0.10 - слабая связь")
print("     0.30 - умеренная связь")
print("     0.50 - сильная связь")
print("\n")

# Создаем объект для расчета мощности
power_analyzer = GofChisquarePower()

# Расчет мощности (аргументы в правильном порядке: effect_size, nobs, alpha, df)

power = power_analyzer.power(effect_size=phi, nobs=n, alpha=0.05, n_bins=observed.size)

print(f"\n МОЩНОСТЬ ТЕСТА:{power:.4f}")
print("   Интерпретация мощности теста:")
print("     < 0.6 (60%) — неадекватная мощность, результаты ненадежны")
print("     0.6-0.7 — низкая мощность")
print("     0.7-0.8 — приемлемая мощность")
print("     ≥ 0.8 — хорошая мощность (стандарт в большинстве исследований)")
print("     0≥ 0.9 — высокая мощность (требуется в клинических испытаниях)")



 РАЗМЕР ЭФФЕКТА (Cohen’s) (φ-коэффициент):
   φ = 0.136
   Интерпретация φ:
     0.10 - слабая связь
     0.30 - умеренная связь
     0.50 - сильная связь



 МОЩНОСТЬ ТЕСТА:0.2268
   Интерпретация мощности теста:
     < 0.6 (60%) — неадекватная мощность, результаты ненадежны
     0.6-0.7 — низкая мощность
     0.7-0.8 — приемлемая мощность
     ≥ 0.8 — хорошая мощность (стандарт в большинстве исследований)
     0≥ 0.9 — высокая мощность (требуется в клинических испытаниях)


In [25]:
print(f'p-value = {p:.4f},\npower(мощность) = {power:.4f}\n')
print('   Интерпретация результатов:')
print('     p < 0.05 & power < 0.6: Значимый результат, но низкая мощность — требуется подтверждение')
print('     p < 0.05 & power >= 0.8: Надежный значимый результат')
print('     p >= 0.05 & power < 0.6: Результат незначим, но тест мог пропустить существующий эффект')
print('     p >= 0.05 & power >= 0.6: Вероятно, эффекта действительно нет или он очень мал')
print('-'*95)

p-value = 0.1194,
power(мощность) = 0.2268

   Интерпретация результатов:
     p < 0.05 & power < 0.6: Значимый результат, но низкая мощность — требуется подтверждение
     p < 0.05 & power >= 0.8: Надежный значимый результат
     p >= 0.05 & power < 0.6: Результат незначим, но тест мог пропустить существующий эффект
     p >= 0.05 & power >= 0.6: Вероятно, эффекта действительно нет или он очень мал
-----------------------------------------------------------------------------------------------


## ВЫВОДЫ
### Подтвержденные гипотезы:
- Наличие расстройств сна влияет на его продолжительность и качество.
- ИМТ является важным фактором, влияющим на наличие расстройств сна.
